In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import pandas as pd

from scipy import stats
from matplotlib import pyplot as plt

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Merge 2018 Data Set to World Cup Data Set and Final Team Rankings

In [2]:
# View Matches dataset
fifa_all = pd.read_csv("WorldCupMatches.csv")

fifa_all.shape

display(fifa_all)

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning Matches Dataset

In [3]:
## Clean unnessary columns
fifa_clean = fifa_all.drop(["Datetime","Stadium", "City", "Attendance", "Referee", "Assistant 1", "Assistant 2", "MatchID"], axis=1)

## Remove trailing NAs
fifa_clean = fifa_clean.dropna(how='all')

# Change old team name of Cote d'Ivoire to Ivory Coast
fifa_clean[['Home Team Name', 'Away Team Name']] = fifa_clean[['Home Team Name', 'Away Team Name']].replace("C�te d'Ivoire", "Ivory Coast")

# Remove extra code chunk of " rn"> "
fifa_clean[['Home Team Name', 'Away Team Name']] = fifa_clean[['Home Team Name', 'Away Team Name']].replace({'rn">':''}, regex=True)

# Filter rows based on years from 1994 to 2014
newdf = (fifa_clean['Year'] >= 1994) & (fifa_clean['Year'] <= 2014)
  
# locate rows and access them using .loc() function
fifa_clean = fifa_clean.loc[newdf]

display(fifa_clean)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials
464,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR
465,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL
466,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI
467,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL
468,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU
...,...,...,...,...,...,...,...,...,...,...,...,...
847,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC
848,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER
849,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG
850,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED


### Now we will add the 2018 Data Set and merge it


In [4]:
fifa_2018 = pd.read_csv("World_cup_2018_matches.csv")

In [5]:
fifa_2018

,Date,Stage,Home,Away,Home_goals,Away_goals,Home_shots,Home_shots_on_target,Home_possession,Home_fouls,...,Away_offsides,Away_corners,Extra_time,Home_extra_goals,Away_extra_goals,Penalties,Home_PK_taken,Home_PK_made,Away_PK_taken,Away_PK_made
0,2018-06-14,Group-1,Russia,Saudi Arabia,5,0,14,7,38,22,...,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-06-15,Group-1,Egypt,Uruguay,0,1,8,3,41,12,...,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-06-15,Group-1,Morocco,Iran,0,1,13,3,68,22,...,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-06-15,Group-1,Portugal,Spain,3,3,9,3,33,12,...,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-06-16,Group-1,France,Australia,2,1,12,5,55,16,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2018-07-07,QFinals,Russia,Croatia,2,2,13,5,35,25,...,0,8,Y,1.0,1.0,Y,5.0,3.0,5.0,4.0
60,2018-07-10,SFinals,France,Belgium,1,0,19,5,36,6,...,1,5,N,NaN,NaN,N,NaN,NaN,NaN,NaN
61,2018-07-11,SFinals,Croatia,England,2,1,22,7,56,23,...,3,4,Y,1.0,0.0,N,NaN,NaN,NaN,NaN
62,2018-07-14,Third-Place,Belgium,England,2,0,12,4,42,11,...,0,5,N,NaN,NaN,N,NaN,NaN,NaN,NaN


### Team Rankings Scraped from fox news

In [6]:
Team_Rankings = pd.read_csv("final_team_rankings.csv")

In [7]:
Team_Rankings

,Unnamed: 0,Year,Host,Champion,Runner Up,Third Place,Teams,Matches Played,Goals Scored,Avg Goals Per Game
0,0,2018,Russia,France,Croatia,Belgium,32,64,169,2.6
1,1,2014,Brazil,Germany,Argentina,Netherlands,32,64,171,2.7
2,2,2010,South Africa,Spain,Netherlands,Germany,32,64,145,2.3
3,3,2006,Germany,Italy,France,Germany,32,64,147,2.3
4,4,2002,"South Korea, Japan",Brazil,Germany,Turkey,32,64,161,2.5
5,5,1998,France,France,Brazil,Croatia,32,64,171,2.7
6,6,1994,United States,Brazil,Italy,Sweden,24,52,141,2.7
7,7,1990,Italy,West Germany,Argentina,Italy,24,52,115,2.2
8,8,1986,Mexico,Argentina,West Germany,France,24,52,132,2.5
9,9,1982,Spain,Italy,West Germany,Poland,24,52,146,2.8


In [8]:
### Now we will first clean the 2018 Dataset by dropping values in it and then reordering the team rankings

In [9]:
fifa_2018.head(10)

,Date,Stage,Home,Away,Home_goals,Away_goals,Home_shots,Home_shots_on_target,Home_possession,Home_fouls,...,Away_offsides,Away_corners,Extra_time,Home_extra_goals,Away_extra_goals,Penalties,Home_PK_taken,Home_PK_made,Away_PK_taken,Away_PK_made
0,2018-06-14,Group-1,Russia,Saudi Arabia,5,0,14,7,38,22,...,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-06-15,Group-1,Egypt,Uruguay,0,1,8,3,41,12,...,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-06-15,Group-1,Morocco,Iran,0,1,13,3,68,22,...,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-06-15,Group-1,Portugal,Spain,3,3,9,3,33,12,...,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-06-16,Group-1,France,Australia,2,1,12,5,55,16,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018-06-16,Group-1,Argentina,Iceland,1,1,27,7,78,10,...,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2018-06-16,Group-1,Peru,Denmark,0,1,17,6,54,10,...,3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2018-06-16,Group-1,Croatia,Nigeria,2,0,11,2,55,20,...,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018-06-17,Group-1,Costa Rica,Serbia,0,1,10,3,53,18,...,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2018-06-17,Group-1,Germany,Mexico,0,1,26,9,67,10,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
### We will now clean this dataset by first dropping all the unneeded columnsabs

In [11]:
fifa_2018_drop =fifa_2018.drop(['Extra_time', 'Home_extra_goals','Away_extra_goals','Penalties','Home_PK_taken','Away_PK_taken','Away_PK_made'],
                        axis = 1, inplace = True) 

In [12]:
fifa_2018

,Date,Stage,Home,Away,Home_goals,Away_goals,Home_shots,Home_shots_on_target,Home_possession,Home_fouls,...,Home_corners,Away_shots,Away_shots_on_target,Away_possession,Away_fouls,Away_yellow,Away_red,Away_offsides,Away_corners,Home_PK_made
0,2018-06-14,Group-1,Russia,Saudi Arabia,5,0,14,7,38,22,...,6,6,0,62,10,1,0,1,2,NaN
1,2018-06-15,Group-1,Egypt,Uruguay,0,1,8,3,41,12,...,0,15,5,59,6,0,0,1,5,NaN
2,2018-06-15,Group-1,Morocco,Iran,0,1,13,3,68,22,...,5,9,2,32,14,3,0,0,2,NaN
3,2018-06-15,Group-1,Portugal,Spain,3,3,9,3,33,12,...,4,13,6,67,10,1,0,3,5,NaN
4,2018-06-16,Group-1,France,Australia,2,1,12,5,55,16,...,5,6,1,45,19,3,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2018-07-07,QFinals,Russia,Croatia,2,2,13,5,35,25,...,6,18,3,65,18,4,0,0,8,3.0
60,2018-07-10,SFinals,France,Belgium,1,0,19,5,36,6,...,4,9,3,64,16,3,0,1,5,NaN
61,2018-07-11,SFinals,Croatia,England,2,1,22,7,56,23,...,8,11,2,44,14,1,0,3,4,NaN
62,2018-07-14,Third-Place,Belgium,England,2,0,12,4,42,11,...,4,15,6,58,4,2,0,0,5,NaN


In [13]:
fifa_2018_dr0p_columns =      fifa_2018.drop(columns=['Home_PK_made'])

In [14]:
fifa_2018_dr0p_columns.isna().sum()

Date                    0
Stage                   0
Home                    0
Away                    0
Home_goals              0
Away_goals              0
Home_shots              0
Home_shots_on_target    0
Home_possession         0
Home_fouls              0
Home_yellow             0
Home_red                0
Home_offsides           0
Home_corners            0
Away_shots              0
Away_shots_on_target    0
Away_possession         0
Away_fouls              0
Away_yellow             0
Away_red                0
Away_offsides           0
Away_corners            0
dtype: int64

In [15]:
#No null values in the 2018 data set so we can merge it with existing world cups but we might have to change the names.

In [16]:
##First we have to reorder team rankings and we will first need to drop a column

In [17]:
Team_Rankings

,Unnamed: 0,Year,Host,Champion,Runner Up,Third Place,Teams,Matches Played,Goals Scored,Avg Goals Per Game
0,0,2018,Russia,France,Croatia,Belgium,32,64,169,2.6
1,1,2014,Brazil,Germany,Argentina,Netherlands,32,64,171,2.7
2,2,2010,South Africa,Spain,Netherlands,Germany,32,64,145,2.3
3,3,2006,Germany,Italy,France,Germany,32,64,147,2.3
4,4,2002,"South Korea, Japan",Brazil,Germany,Turkey,32,64,161,2.5
5,5,1998,France,France,Brazil,Croatia,32,64,171,2.7
6,6,1994,United States,Brazil,Italy,Sweden,24,52,141,2.7
7,7,1990,Italy,West Germany,Argentina,Italy,24,52,115,2.2
8,8,1986,Mexico,Argentina,West Germany,France,24,52,132,2.5
9,9,1982,Spain,Italy,West Germany,Poland,24,52,146,2.8


In [18]:
final_team_rankings =Team_Rankings.sort_values("Year")

In [19]:
final_team_rankings

,Unnamed: 0,Year,Host,Champion,Runner Up,Third Place,Teams,Matches Played,Goals Scored,Avg Goals Per Game
20,20,1930,Uruguay,Uruguay,Argentina,United States,13,16,70,3.6
19,19,1934,Italy,Italy,Czechoslovakia,Germany,16,17,70,4.1
18,18,1938,France,Italy,Hungary,Brazil,15,18,84,4.7
17,17,1950,Brazil,Uruguay,Brazil,Sweden,13,22,88,4.0
16,16,1954,Switzerland,West Germany,Hungary,Austria,16,26,140,5.4
15,15,1958,Sweden,Brazil,Sweden,France,16,35,126,3.6
14,14,1962,Chile,Brazil,Czechoslovakia,Chile,16,32,89,2.8
13,13,1966,England,England,West Germany,Portugal,16,32,89,2.8
12,12,1970,Mexico,Brazil,Italy,West Germany,16,32,95,3.0
11,11,1974,West Germany,West Germany,Netherlands,Poland,16,38,97,2.6


In [20]:
##Now we will drop the unnamed column as it is not required.

In [21]:
final_rankings =  final_team_rankings.drop(columns=['Unnamed: 0'])

In [22]:
final_rankings

,Year,Host,Champion,Runner Up,Third Place,Teams,Matches Played,Goals Scored,Avg Goals Per Game
20,1930,Uruguay,Uruguay,Argentina,United States,13,16,70,3.6
19,1934,Italy,Italy,Czechoslovakia,Germany,16,17,70,4.1
18,1938,France,Italy,Hungary,Brazil,15,18,84,4.7
17,1950,Brazil,Uruguay,Brazil,Sweden,13,22,88,4.0
16,1954,Switzerland,West Germany,Hungary,Austria,16,26,140,5.4
15,1958,Sweden,Brazil,Sweden,France,16,35,126,3.6
14,1962,Chile,Brazil,Czechoslovakia,Chile,16,32,89,2.8
13,1966,England,England,West Germany,Portugal,16,32,89,2.8
12,1970,Mexico,Brazil,Italy,West Germany,16,32,95,3.0
11,1974,West Germany,West Germany,Netherlands,Poland,16,38,97,2.6


In [23]:
#Now we need to merge this data set with original data set but before that rename the column names for the 2018 data set.

In [24]:
fifa_cup_2018 = fifa_2018_dr0p_columns

In [25]:
fifa_cup_2018

,Date,Stage,Home,Away,Home_goals,Away_goals,Home_shots,Home_shots_on_target,Home_possession,Home_fouls,...,Home_offsides,Home_corners,Away_shots,Away_shots_on_target,Away_possession,Away_fouls,Away_yellow,Away_red,Away_offsides,Away_corners
0,2018-06-14,Group-1,Russia,Saudi Arabia,5,0,14,7,38,22,...,3,6,6,0,62,10,1,0,1,2
1,2018-06-15,Group-1,Egypt,Uruguay,0,1,8,3,41,12,...,1,0,15,5,59,6,0,0,1,5
2,2018-06-15,Group-1,Morocco,Iran,0,1,13,3,68,22,...,0,5,9,2,32,14,3,0,0,2
3,2018-06-15,Group-1,Portugal,Spain,3,3,9,3,33,12,...,1,4,13,6,67,10,1,0,3,5
4,2018-06-16,Group-1,France,Australia,2,1,12,5,55,16,...,0,5,6,1,45,19,3,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2018-07-07,QFinals,Russia,Croatia,2,2,13,5,35,25,...,1,6,18,3,65,18,4,0,0,8
60,2018-07-10,SFinals,France,Belgium,1,0,19,5,36,6,...,1,4,9,3,64,16,3,0,1,5
61,2018-07-11,SFinals,Croatia,England,2,1,22,7,56,23,...,1,8,11,2,44,14,1,0,3,4
62,2018-07-14,Third-Place,Belgium,England,2,0,12,4,42,11,...,1,4,15,6,58,4,2,0,0,5


In [26]:
fifa_clean

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials
464,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR
465,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL
466,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI
467,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL
468,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU
...,...,...,...,...,...,...,...,...,...,...,...,...
847,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC
848,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER
849,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG
850,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED


In [27]:
### Rankings is applied to the data set. Now rename the columns for the data set of 20i8 then append it.

In [28]:
fifa_cup_2018.rename(columns={'Date': 'Year',
                   'Home': 'Home Team Name','Away':'Away Team Goals','Home_goals':'Home Team Goals','Away_goals':'Away Team Goals'},
          inplace=True)
# Make sure you set inplace to True if you want the change
# to be applied to the dataframe

In [29]:
fifa_cup_2018

,Year,Stage,Home Team Name,Away Team Goals,Home Team Goals,Away Team Goals,Home_shots,Home_shots_on_target,Home_possession,Home_fouls,...,Home_offsides,Home_corners,Away_shots,Away_shots_on_target,Away_possession,Away_fouls,Away_yellow,Away_red,Away_offsides,Away_corners
0,2018-06-14,Group-1,Russia,Saudi Arabia,5,0,14,7,38,22,...,3,6,6,0,62,10,1,0,1,2
1,2018-06-15,Group-1,Egypt,Uruguay,0,1,8,3,41,12,...,1,0,15,5,59,6,0,0,1,5
2,2018-06-15,Group-1,Morocco,Iran,0,1,13,3,68,22,...,0,5,9,2,32,14,3,0,0,2
3,2018-06-15,Group-1,Portugal,Spain,3,3,9,3,33,12,...,1,4,13,6,67,10,1,0,3,5
4,2018-06-16,Group-1,France,Australia,2,1,12,5,55,16,...,0,5,6,1,45,19,3,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2018-07-07,QFinals,Russia,Croatia,2,2,13,5,35,25,...,1,6,18,3,65,18,4,0,0,8
60,2018-07-10,SFinals,France,Belgium,1,0,19,5,36,6,...,1,4,9,3,64,16,3,0,1,5
61,2018-07-11,SFinals,Croatia,England,2,1,22,7,56,23,...,1,8,11,2,44,14,1,0,3,4
62,2018-07-14,Third-Place,Belgium,England,2,0,12,4,42,11,...,1,4,15,6,58,4,2,0,0,5


In [30]:
### Now we merge all data sets

In [32]:
### Visualisations and Insights for the data set.

In [33]:
gold = fifa_clean["Winner"]
silver = fifa_clean["Runners-Up"]
bronze = fifa_clean["Third"]

gold_count = pd.DataFrame.from_dict(gold.value_counts())
silver_count = pd.DataFrame.from_dict(gold.value_counts())
bronze_count = pd.DataFrame.from_dict(gold.value_counts())
podium_count = gold_count.join(silver_count, how="outer").join(bronze_count, how='outer')
podium_count = podium_count.fillna(0)
podium_count.columns = ['Winner','Second','Third']
podium_count = podium_count.astype('int64')
podium_count = podium_count.sort_values(by=['Winner','Second','Third'], ascending=False)
podium_count.plot(y=['Winner','Second','Third'], kind="bar", color=['gold','silver','brown'], figsize=(15,6), fontsize=14,
                 width=0.8, align='centre')
plt.xlabel('Countries')
plt.ylabel('Number of podium')
plt.title('Number of podium by country ')

KeyError: 'Winner'